In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './picture'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import os

# = = = = = = = = = = = = = = =

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('./')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import PIL

# = = = = = = = = = = = = = = =

from io import BytesIO

# = = = = = = = = = = = = = = =

from openpyxl.drawing.image import Image

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

size, quality = 500, '100%'

# = = = = = = = = = = = = = = =

directory_in = './crawler'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store', '0.null.txt'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    wb = load_workbook(f'{directory_in}/{file}')
    ws = wb.worksheets[0]

    for column in range(1, ws.max_column + 1):
        if ws.cell(1, column).value == 'Json_Src':
            column_src = column
        elif ws.cell(1, column).value == 'Picture':
            column_pic = column

    count_remain = ws.max_row - 1
    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_index):
        global ws, count_remain

        crawler_status = False

        # = = = = = = = = = = = = = = =

        try:
            src = json.loads(ws.cell(crawler_index, column_src).value).get('0', '')

            # = = = = = = = = = = = = = = =

            if src == '':
                count_retry, crawler_status = 0, True
            else:
                count_retry = 0
                while True:
                    count_retry += 1

                    try:
                        resp = requests.get(src,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(5, 15))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                image = PIL.Image.open(BytesIO(resp.content)).convert('RGB')

                # = = = = = = = = = = = = = = =

                scale = size / max(image.width, image.height)
                image = image.resize((int(image.width * scale), int(image.height * scale)), PIL.Image.Resampling.LANCZOS)

                # = = = = = = = = = = = = = = =

                buffer = BytesIO()
                if quality == '100%':
                    image.save(buffer, format="PNG")
                else:
                    image.save(buffer, format="JPEG", quality=int(compress.removesuffix('%')))
                buffer.seek(0)

                pic = Image(buffer)

                # = = = = = = = = = = = = = = =

                with semaphore_correct:
                    ws.add_image(pic, f'{get_column_letter(column_pic)}{crawler_index}')

                # = = = = = = = = = = = = = = =

                crawler_status = True
        except:
            pass

        # = = = = = = = = = = = = = = =

        if count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {src}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, range(2, ws.max_row + 1))

    print('输出ing...')
    print()
    wb.save(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''')

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[True] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/15 20:04:47] - https://static.dormanproducts.com/images/product/medium/924-218-001.JPG

[True] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/15 20:04:47] - https://static.dormanproducts.com/images/product/medium/924-234-007.JPG

[True] - [请求1次] - [剩余7条] - [crawler_1（50.00%）] - [2024/10/15 20:04:47] - https://static.dormanproducts.com/images/product/medium/924-219-001.JPG

[True] - [请求1次] - [剩余6条] - [crawler_1（50.00%）] - [2024/10/15 20:04:47] - https://static.dormanproducts.com/images/product/medium/924-220-001.JPG

[True] - [请求1次] - [剩余5条] - [crawler_1（50.00%）] - [2024/10/15 20:04:50] - https://static.dormanproducts.com/images/product/medium/924-659-007.JPG

[True] - [请求1次] - [剩余4条] - [crawler_1（50.00%）] - [2024/10/15 20:04:50] - https://static.dormanproducts.com/images/product/medium/924-375-007.JPG

[True] - [请求1次] - [剩余3条] - [crawler_1（50.00%）] - [2024/10/15 20:04:51] - https://static.dormanproducts.com/images/pr

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3869.28it/s]

Done ~
